In [23]:
import tkinter as tk
from PIL import Image, ImageTk 
from tkinter import filedialog
import os

Einfuegung des tKinter, mit einsetyung des 

In [44]:


# Erstelle das Hauptfenster
root = tk.Tk()
root.geometry("550x300")
root.title("Speakeasy v0.1")
root.resizable(width=True, height=True)

# Lade das Bild und überprüfe den Pfad
logo_path = r"C:\Users\Admin\Desktop\Speakeasy\logo_Speakeasy_1080p.png"


logo = Image.open(logo_path)
logo = logo.resize((200, 200))  # Größe anpassen
logo2 = ImageTk.PhotoImage(logo)

        # Label mit Bild erstellen
label = tk.Label(root,text= 'Wilkommen bei Speakeasy.\n Bitte waehlen sie eine Datei aus, um den Text zu extrahieren', image=logo2, compound= 'top')
label.logo2 = logo2 
label.pack(anchor=tk.CENTER)

# Button zur Auswahl einer Datei
button = tk.Button(root, text="Datei auswählen")
button.pack()

# Tkinter-Hauptloop starten
root.mainloop()


TclError: image "pyimage26" doesn't exist